In [8]:
import numpy as np
import scipy.sparse
import numba
import pandas as pd
import stopwordsiso
import Stemmer
import re
from tqdm import tqdm
import collections

In [9]:
class WordToBase:
    """
    Class to convert words to their base form
    """
    def __init__(self, lang):
        self.lang = lang

    def get_stemmer(self, single=False):
        try:
            if single:
                return Stemmer.Stemmer(self.lang).stemWord
            return Stemmer.Stemmer(self.lang).stemWords
        except Exception as e:
            print(f'Inbuilt Stemmer does not exist for {self.lang}, creating one!')
            stemmer = self.create_stemmer(single)
        return stemmer
    
    # to be overwritten -> korean doesn't have stemmer from PyStemmer
    def create_stemmer(self, single=False):
        if single:
            return Stemmer.Stemmer('en').stemWord
        return Stemmer.Stemmer('en').stemWords


In [10]:
class TextProcessor:
    """
    Class to preprocess the corpus and queries
    """
    def __init__(self, lang, stopwords=None):
        self.lang = lang
        self.word_to_base = {}
        self.base_to_baseidx = {}
        self.word_to_wordidx = {}
        self.remove_punct = re.compile(r'(?u)\b\w\w+\b') # This is what sklearn uses
        if stopwords is None:
            self.stopwords = set(self.get_stopwords(lang))
        else:
            self.stopwords = stopwords

    def get_stopwords(self, lang):
        """
        Stopwords for the corresponding language
        """
        return stopwordsiso.stopwords(lang)
        

    def preprocess_corpus(self, corpus):
        """
        Given a corpus, which is a list of documents, we do the following for each document:
            - Convert document to lowercase
            - For all words in the document, we remove punctuations
            - If the word is a stopword, we discard it
            - Each word is converted to its base form using the stemmer
            - We create several mappings:
                • self.word_to_base: word -> base_word
                • self.base_to_baseidx: base_word -> base_idx (serves as the vocabulary)
                • self.word_to_wordidx: word -> base_idx
        We then return a list of lists, where each list is a document, and each element in the list is the base_idx of the word
        """
        stemmer = WordToBase(self.lang).get_stemmer(single=True)
        corpus_token_indices = []
        for doc in tqdm(corpus):
            document_token_indices = []
            doc = doc.lower()
            words = list(self.remove_punct.findall(doc))

            for word in words:
                # if we re-encounter the word, we don't need to recompute the base word
                if word in self.word_to_wordidx:
                    document_token_indices.append(self.word_to_wordidx[word])
                    continue
                # if we encounter a stopword, we discard it
                # Note, this if condition is 2nd as it improves performance
                if word in self.stopwords:
                    continue
                
                # if we have already computed the base word, we use it
                if word in self.word_to_base:
                    base_word = self.word_to_base[word]
                # otherwise, we compute the base word
                else:
                    base_word = stemmer(word)
                    self.word_to_base[word] = base_word
                # if we have already computed the base_idx, we use it
                if base_word in self.base_to_baseidx:
                    base_idx = self.base_to_baseidx[base_word]
                    self.word_to_wordidx[word] = base_idx
                    document_token_indices.append(base_idx)
                # else we compute the base_idx and update the mappings
                else:
                    base_idx = len(self.base_to_baseidx)
                    self.base_to_baseidx[base_word] = base_idx
                    self.word_to_wordidx[word] = base_idx
                    document_token_indices.append(base_idx)
            corpus_token_indices.append(document_token_indices)

        return corpus_token_indices, self.base_to_baseidx
    
    def preprocess_queries(self, queries):
        """ 
        Given a list of queries, we do the following for each query:
            - Convert query to lowercase
            - For all words in the query, we remove punctuations
            - If the word is a stopword, we discard it
            - We first form a collection of all the words in the queries
            - We then compute the base form of each word
            - Then we update all mappings, so that we can convert the query words to base_idx
        """
        query_token_ids = []
        word_to_idx = {}
        stemmer = WordToBase(self.lang).get_stemmer(single=False)
        for query in queries:
            query = query.lower() 
            words = self.remove_punct.findall(query)
            query_ids = []
            for word in words:
                # If we encounter a stopword, we discard it
                if word in self.stopwords:
                    continue
                # If we have not seen the word before, we update the mappings
                if word not in word_to_idx:
                    word_to_idx[word] = len(word_to_idx)
                
                # Append the word_idx to the query_ids
                word_idx = word_to_idx[word]
                query_ids.append(word_idx)
            query_token_ids.append(query_ids)

        # After the above computation, we have a collection of all query words
        # Note that, it is not trivial to use the corpus vocabulary, since queries can have unseen words
        # We now compute the base form of each word and update the mappings

        # We first get the unique words in the queries
        unique_words = list(word_to_idx.keys())
        # We then compute the base form of each word
        base_words = stemmer(unique_words)
        unique_base_words = set(base_words)

        # We generate the base_word -> base_idx mapping
        unique_base_to_baseidx = {x:i for (i,x) in enumerate(unique_base_words)}

        # We create a mapping from word_idx -> base_idx
        wordidx_to_baseidx = {word_to_idx[word]:unique_base_to_baseidx[base] for (word, base) in zip(unique_words, base_words)}

        # Finally, we convert all to corresponding base words
        for i, query_tokens in enumerate(query_token_ids):
            query_token_ids[i] = [wordidx_to_baseidx[x] for x in query_tokens]

        return query_token_ids, unique_base_to_baseidx


In [68]:
@numba.njit()
def sift_down(values, indices, startpos, pos):
    new_value = values[pos]
    new_index = indices[pos]
    while pos > startpos:
        parentpos = (pos - 1) >> 1
        parent_value = values[parentpos]
        if new_value < parent_value:
            values[pos] = parent_value
            indices[pos] = indices[parentpos]
            pos = parentpos
            continue
        break
    values[pos] = new_value
    indices[pos] = new_index


@numba.njit()
def sift_up(values, indices, pos, length):
    startpos = pos
    new_value = values[pos]
    new_index = indices[pos]
    childpos = 2 * pos + 1
    while childpos < length:
        rightpos = childpos + 1
        if rightpos < length and values[rightpos] < values[childpos]:
            childpos = rightpos
        values[pos] = values[childpos]
        indices[pos] = indices[childpos]
        pos = childpos
        childpos = 2 * pos + 1
    values[pos] = new_value
    indices[pos] = new_index
    sift_down(values, indices, startpos, pos)


@numba.njit()
def heap_push(values, indices, value, index, length):
    values[length] = value
    indices[length] = index
    sift_down(values, indices, 0, length)


@numba.njit()
def heap_pop(values, indices, length):
    return_value = values[0]
    return_index = indices[0]
    last_value = values[length - 1]
    last_index = indices[length - 1]
    values[0] = last_value
    indices[0] = last_index
    sift_up(values, indices, 0, length - 1)
    return return_value, return_index

@numba.njit()
def parallel_topk(array, topk):
    n = len(array)
    if topk > n:
        topk = n

    values = np.zeros(topk, dtype=array.dtype)  # aka scores
    indices = np.zeros(topk, dtype=np.int64)
    length = 0

    for i, value in enumerate(array):
        if length < topk:
            heap_push(values, indices, value, i, length)
            length += 1
        else:
            if value > values[0]:
                values[0] = value
                indices[0] = i
                sift_up(values, indices, 0, length)

    sorted_indices = np.flip(np.argsort(values))
    indices = indices[sorted_indices]
    values = values[sorted_indices]

    return values, indices

@numba.njit
def query_token_score(single_query_tokens, score_indptr, indices, data, num_corpus):
    start = score_indptr[single_query_tokens]
    end = score_indptr[single_query_tokens + 1]
    scores = np.zeros(num_corpus, dtype=np.float32)
    for j in range(len(single_query_tokens)):
        _s, _e = start[j], end[j]
        for k in range(_s, _e):
            scores[indices[k]] += data[k]

    return scores

@numba.njit(parallel=True)
def all_query_token_score(query_ptrs, query_tokens_ids_flat, topk, score_indptr, indices, data, num_corpus):
    topk_scores = np.zeros((len(query_ptrs)-1, topk), dtype=np.float32)
    topk_indices = np.zeros((len(query_ptrs)-1, topk), dtype=np.int64)

    for i in numba.prange(len(query_ptrs) - 1):
        single_query_tokens = query_tokens_ids_flat[query_ptrs[i]:query_ptrs[i+1]]
        single_query_score = query_token_score(single_query_tokens, score_indptr, indices, data, num_corpus)
        topk_scores_sing, topk_indices_sing = parallel_topk(single_query_score, topk=topk)
        topk_scores[i] = topk_scores_sing
        topk_indices[i] = topk_indices_sing

    return topk_scores, topk_indices


In [60]:

class TFIDF:
    def __init__(self, corpus, save_path=None):
        if save_path is not None:
            self.load(save_path)
        else:
            self.corpus = corpus 
            self.num_corpus = len(self.corpus)

    def save(self, save_path):
        """
        Save the TF-IDF scores to the disk
        """
        path = f'{save_path}/TFIDF'
        np.save(f'{path}/scores_data.npy', self.score_data)
        np.save(f'{path}/scores_indices.npy', self.score_indices)
        np.save(f'{path}/scores_indptr.npy', self.score_indptr)
        with open(f'{path}/token_map.json', 'w') as f:
            json.dump(self.token_map, f)
        with open(f'params.json', 'w') as f:
            json.dump({'num_corpus': self.num_corpus, 'num_unique': self.num_unique}, f)
        
    def load(self, save_path):
        """ 
        Load the TF-IDF scores from the disk
        """
        path = f'{save_path}/TFIDF'
        self.score_data = np.load(f'{path}/scores_data.npy')
        self.score_indices = np.load(f'{path}/scores_indices.npy')
        self.score_indptr = np.load(f'{path}/scores_indptr.npy')
        with open(f'{path}/token_map.json', 'r') as f:
            self.token_map = json.load(f)
        with open(f'params.json', 'r') as f:
            params = json.load(f)
            self.num_corpus = params['num_corpus']
            self.num_unique = params

    def calculate_scores(self, corpus_tokens, token_map):
        """
        Given the corpus tokens, and the token_map, we calculate the TF-IDF scores
        """

        self.token_map = token_map
        unique_tokens = list(token_map.values())
        self.num_unique = len(unique_tokens)

        # Calculating Document Frequency
        set_unique_tokens = set(unique_tokens)
        DF = {x: 0 for x in set_unique_tokens}
        for document_tokens in corpus_tokens:
            tokens_present = set_unique_tokens.intersection(document_tokens)
            for token in tokens_present:
                DF[token] += 1

        self.IDF = self.calculate_idf(DF)

        tfidf_scores = np.empty(sum(DF.values()), dtype=np.float32)
        doc_indices = np.empty(sum(DF.values()), dtype=np.int64)
        word_indices = np.empty(sum(DF.values()), dtype=np.int64)

        # calculate tf-idf for each term
        ptr = 0
        for i, doc_tokens in enumerate(corpus_tokens):
            num_tokens = len(doc_tokens)
            doc_token_counts = collections.Counter(doc_tokens)
            doc_token_indices = np.array(list(doc_token_counts.keys()), dtype=np.int64)
            doc_token_counts = np.array(list(doc_token_counts.values()), dtype=np.float32)
            
            weighted_tf = doc_token_counts / num_tokens
            tfidf = weighted_tf * self.IDF[doc_token_indices]
            tfidf_scores[ptr:ptr+len(doc_token_indices)] = tfidf
            doc_indices[ptr:ptr+len(doc_token_indices)] = i
            word_indices[ptr:ptr+len(doc_token_indices)] = doc_token_indices
            ptr += len(doc_token_indices)

        tfidf_matrix = scipy.sparse.csc_matrix((tfidf_scores, (doc_indices, word_indices)), shape=(self.num_corpus, self.num_unique), dtype=np.float32)
        self.tfidf_data = tfidf_matrix.data
        self.tfidf_indices = tfidf_matrix.indices
        self.tfidf_indptr = tfidf_matrix.indptr

    def calculate_idf(self, DF):
        """
        Calculate the inverse document frequency of each token
        """
        IDF = np.zeros(self.num_unique, dtype=np.float32)
        for token, _ in DF.items():
            IDF[token] = np.log((1 + self.num_corpus) / (1 + DF[token])) + 1
        return IDF
    
    def parallel_search(self, query_tokens, query_token_map, topk=10, num_threads=10):
        """
        Given the query tokens and the query_token_map, we calculate the topk documents for each query
        """
        inverse_query_token_map = {v: k for k, v in query_token_map.items()}
        query_tokens = [[inverse_query_token_map[x] for x in query] for query in query_tokens]
        query_token_ids = [[self.token_map[x] for x in query if x in self.token_map] for query in query_tokens]
        query_tokens_ids_flat = np.concatenate(query_token_ids).astype(np.int64)
        query_ptrs = np.cumsum([0] + [len(x) for x in query_token_ids], dtype=np.int64)

        numba.set_num_threads(num_threads)
        topk_scores, topk_indices = all_query_token_score(query_ptrs, query_tokens_ids_flat, topk, self.tfidf_indptr, self.tfidf_indices, self.tfidf_data, self.num_corpus)
        numba.set_num_threads(1)
        return topk_indices, topk_scores



In [73]:
class BM25_V1(TFIDF):
    """
    In BM25_V1, we set the IDF[t] = log((num_corpus - DF[t] + 0.5)/(DF[t] + 0.5) + 1)
    """
    def __init__(self, corpus, k1, b, save_path=None):
        self.k1 = k1
        self.b = b
        super().__init__(corpus, save_path)

    def save(self, save_path):
        """
        Save the TF-IDF scores to the disk
        """
        path = f'{save_path}/BM25_V1'
        np.save(f'{path}/scores_data.npy', self.score_data)
        np.save(f'{path}/scores_indices.npy', self.score_indices)
        np.save(f'{path}/scores_indptr.npy', self.score_indptr)
        with open(f'{path}/token_map.json', 'w') as f:
            json.dump(self.token_map, f)
        with open(f'params.json', 'w') as f:
            json.dump({'num_corpus': self.num_corpus, 'num_unique': self.num_unique, 'k1': self.k1, 'b': self.b}, f)
        
    def load(self, save_path):
        """ 
        Load the TF-IDF scores from the disk
        """
        path = f'{save_path}/BM25_V1'
        self.score_data = np.load(f'{path}/scores_data.npy')
        self.score_indices = np.load(f'{path}/scores_indices.npy')
        self.score_indptr = np.load(f'{path}/scores_indptr.npy')
        with open(f'{path}/token_map.json', 'r') as f:
            self.token_map = json.load(f)
        with open(f'params.json', 'r') as f:
            params = json.load(f)
            self.num_corpus = params['num_corpus']
            self.num_unique = params['num_unique']
            self.k1 = params['k1']
            self.b = params['b']

    def calculate_scores(self, corpus_tokens, token_map):
        """
        Given the corpus tokens, and the token_map, we calculate the BM25 scores
        """

        self.token_map = token_map
        unique_tokens = list(token_map.values())
        self.num_unique = len(unique_tokens)
        L_avg = sum([len(x) for x in corpus_tokens]) / len(corpus_tokens)

        # Calculating Document Frequency
        set_unique_tokens = set(unique_tokens)
        DF = {x: 0 for x in set_unique_tokens}
        for document_tokens in corpus_tokens:
            tokens_present = set_unique_tokens.intersection(document_tokens)
            for token in tokens_present:
                DF[token] += 1

        self.IDF = self.calculate_idf(DF)

        tfidf_scores = np.empty(sum(DF.values()), dtype=np.float32)
        doc_indices = np.empty(sum(DF.values()), dtype=np.int64)
        word_indices = np.empty(sum(DF.values()), dtype=np.int64)

        # calculate tf-idf for each term
        ptr = 0
        for i, doc_tokens in enumerate(corpus_tokens):
            num_tokens = len(doc_tokens)
            doc_token_counts = collections.Counter(doc_tokens)
            doc_token_indices = np.array(list(doc_token_counts.keys()), dtype=np.int64)
            doc_token_counts = np.array(list(doc_token_counts.values()), dtype=np.float32)
            
            weighted_tf = doc_token_counts / (doc_token_counts + self.k1 * (1 - self.b*(1 - num_tokens/L_avg)))
            tfidf = weighted_tf * self.IDF[doc_token_indices]
            tfidf_scores[ptr:ptr+len(doc_token_indices)] = tfidf
            doc_indices[ptr:ptr+len(doc_token_indices)] = i
            word_indices[ptr:ptr+len(doc_token_indices)] = doc_token_indices
            ptr += len(doc_token_indices)

        tfidf_matrix = scipy.sparse.csc_matrix((tfidf_scores, (doc_indices, word_indices)), shape=(self.num_corpus, self.num_unique), dtype=np.float32)
        self.tfidf_data = tfidf_matrix.data
        self.tfidf_indices = tfidf_matrix.indices
        self.tfidf_indptr = tfidf_matrix.indptr

    def calculate_idf(self, DF):
        """
        Calculate the inverse document frequency of each token
        """
        IDF = np.zeros(self.num_unique, dtype=np.float32)
        for token, _ in DF.items():
            IDF[token] = np.log((self.num_corpus - DF[token] + 0.5)/(DF[token] + 0.5) + 1)
        return IDF

In [77]:
class BM25_V2(BM25_V1):
    """
    In BM25_V2, we set the IDF[t] = log((1 + num_corpus)/(1 + DocFreq[t])) + 1
    """
    def __init__(self, corpus, k1, b, save_path=None):
        self.k1 = k1
        self.b = b
        super().__init__(corpus, k1, b, save_path)

    def save(self, save_path):
        """
        Save the TF-IDF scores to the disk
        """
        path = f'{save_path}/BM25_V2'
        np.save(f'{path}/scores_data.npy', self.score_data)
        np.save(f'{path}/scores_indices.npy', self.score_indices)
        np.save(f'{path}/scores_indptr.npy', self.score_indptr)
        with open(f'{path}/token_map.json', 'w') as f:
            json.dump(self.token_map, f)
        with open(f'params.json', 'w') as f:
            json.dump({'num_corpus': self.num_corpus, 'num_unique': self.num_unique, 'k1': self.k1, 'b': self.b}, f)
        
    def load(self, save_path):
        """ 
        Load the TF-IDF scores from the disk
        """
        path = f'{save_path}/BM25_V2'
        self.score_data = np.load(f'{path}/scores_data.npy')
        self.score_indices = np.load(f'{path}/scores_indices.npy')
        self.score_indptr = np.load(f'{path}/scores_indptr.npy')
        with open(f'{path}/token_map.json', 'r') as f:
            self.token_map = json.load(f)
        with open(f'params.json', 'r') as f:
            params = json.load(f)
            self.num_corpus = params['num_corpus']
            self.num_unique = params['num_unique']
            self.k1 = params['k1']
            self.b = params['b']

    def calculate_idf(self, DF):
        """
        Calculate the inverse document frequency of each token
        """
        IDF = np.zeros(self.num_unique, dtype=np.float32)
        for token, _ in DF.items():
            IDF[token] = np.log((1 + self.num_corpus) / (1 + DF[token])) + 1
        return IDF


In [6]:

import json
corpus = json.load(open('../dis-project-1-document-retrieval/corpus.json/corpus.json', 'r'))

In [7]:
STOPWORDS_FRENCH = (
    "ai",
    "aie",
    "aient",
    "aies",
    "ait",
    "as",
    "au",
    "aura",
    "aurai",
    "auraient",
    "aurais",
    "aurait",
    "auras",
    "aurez",
    "auriez",
    "aurions",
    "aurons",
    "auront",
    "aux",
    "avaient",
    "avais",
    "avait",
    "avec",
    "avez",
    "aviez",
    "avions",
    "avons",
    "ayant",
    "ayante",
    "ayantes",
    "ayants",
    "ayez",
    "ayons",
    "c",
    "ce",
    "ces",
    "d",
    "dans",
    "de",
    "des",
    "du",
    "elle",
    "en",
    "es",
    "est",
    "et",
    "eu",
    "eue",
    "eues",
    "eurent",
    "eus",
    "eusse",
    "eussent",
    "eusses",
    "eussiez",
    "eussions",
    "eut",
    "eux",
    "eûmes",
    "eût",
    "eûtes",
    "furent",
    "fus",
    "fusse",
    "fussent",
    "fusses",
    "fussiez",
    "fussions",
    "fut",
    "fûmes",
    "fût",
    "fûtes",
    "il",
    "ils",
    "j",
    "je",
    "l",
    "la",
    "le",
    "les",
    "leur",
    "lui",
    "m",
    "ma",
    "mais",
    "me",
    "mes",
    "moi",
    "mon",
    "même",
    "n",
    "ne",
    "nos",
    "notre",
    "nous",
    "on",
    "ont",
    "ou",
    "par",
    "pas",
    "pour",
    "qu",
    "que",
    "qui",
    "s",
    "sa",
    "se",
    "sera",
    "serai",
    "seraient",
    "serais",
    "serait",
    "seras",
    "serez",
    "seriez",
    "serions",
    "serons",
    "seront",
    "ses",
    "soient",
    "sois",
    "soit",
    "sommes",
    "son",
    "sont",
    "soyez",
    "soyons",
    "suis",
    "sur",
    "t",
    "ta",
    "te",
    "tes",
    "toi",
    "ton",
    "tu",
    "un",
    "une",
    "vos",
    "votre",
    "vous",
    "y",
    "à",
    "étaient",
    "étais",
    "était",
    "étant",
    "étante",
    "étantes",
    "étants",
    "étiez",
    "étions",
    "été",
    "étée",
    "étées",
    "étés",
    "êtes",
)

STOPWORDS_EN = (
    "a",
    "an",
    "and",
    "are",
    "as",
    "at",
    "be",
    "but",
    "by",
    "for",
    "if",
    "in",
    "into",
    "is",
    "it",
    "no",
    "not",
    "of",
    "on",
    "or",
    "such",
    "that",
    "the",
    "their",
    "then",
    "there",
    "these",
    "they",
    "this",
    "to",
    "was",
    "will",
    "with",
)

In [64]:
french_text = [x['text'] for x in corpus if x['lang'] == 'fr']
french_corpus = [x for x in corpus if x['lang'] == 'fr']
P = TextProcessor('french', stopwords=STOPWORDS_EN)
corpus_tokens, corpus_map = P.preprocess_corpus(french_text)

100%|██████████| 10676/10676 [00:16<00:00, 634.32it/s]


In [65]:
dev_df = pd.read_csv('../dis-project-1-document-retrieval/dev.csv')
my_dev_df = dev_df[dev_df['lang'] == 'fr']
my_dev_set_queries = my_dev_df['query'].tolist()
my_dev_set_positive_docs = my_dev_df['positive_docs'].tolist()
query_tokens, query_map = P.preprocess_queries(my_dev_set_queries)

In [66]:
def idx_to_docid(idx_list_list, french_c_with_id):
    res = []
    for idx_list in idx_list_list:
        res.append(list(map(lambda x: french_c_with_id[x]['docid'], idx_list)))
    return res
def recall_at_10(positive_docs, top_10_ids):
    recall = []
    for positive_doc, top_10_id in zip(positive_docs, top_10_ids):
        recall.append(positive_doc in top_10_id)

    print(np.mean(recall))
french_c_with_id = [x for x in corpus if x['lang'] == 'fr']

In [69]:
tf_idf = TFIDF(french_text)
tf_idf.calculate_scores(corpus_tokens, corpus_map)
results, scores = tf_idf.parallel_search(query_tokens, query_map, topk=10, num_threads=1)
s = idx_to_docid(results, french_c_with_id)
recall_at_10(my_dev_set_positive_docs, s)

0.265


In [74]:
tf_idf = BM25_V1(french_text, k1=1.5, b=0.75)
tf_idf.calculate_scores(corpus_tokens, corpus_map)
results, scores = tf_idf.parallel_search(query_tokens, query_map, topk=10, num_threads=1)
s = idx_to_docid(results, french_c_with_id)
recall_at_10(my_dev_set_positive_docs, s)

0.895


In [78]:
tf_idf = BM25_V2(french_text, k1=1.5, b=0.75, save_path=None)
tf_idf.calculate_scores(corpus_tokens, corpus_map)
results, scores = tf_idf.parallel_search(query_tokens, query_map, topk=10, num_threads=1)
s = idx_to_docid(results, french_c_with_id)
recall_at_10(my_dev_set_positive_docs, s)

0.9


In [1]:
class A:
    def __init__(self, x):
        self.x = x
        self.mul()
    def mul(self):
        print(self.x * 2)
class B(A):
    def __init__(self, x):
        super().__init__(x)
    def mul(self):
        print(self.x * 3)

b = B(2)

6
